# Advanced: Plotting drug binding hotspots on the protein

This is an example of some of the more advanced things one can do with Contact Map Explorer. The system here is a drug molecule (a single residue) binding to a protein. So our query is the drug molecule residue, and the haystack is the protein (heavy atoms for both).

Since there's only one residue in the query, that residue is extraneous information. When we remove that, we're left with the fraction of the trajectory that the drug spends in contact with each protein residue -- in other words, this will tell us where the hotspots in the binding event are. But the 3-dimensional nature of proteins means sequence alone doesn't show you the hotspots, you need a 3D representation of the structure.

For that 3D representation, we'll go to [NGLView](https://github.com/arose/nglview), an excellent tool for visualizing 3D molecular structures in Jupyter notebooks. We'll use the values from our contact map to set the color of the residues in the NGLView visualization.

Note that NGLView is not a requirement for Contact Map Explorer, so you may need to install it separately. It can be easily installed with conda; see [the NGLView GitHub page](https://github.com/arose/nglview) for details.

The first part of this is exactly like the contact concurrences example:

In [1]:
from __future__ import print_function
import numpy as np

from contact_map import ContactFrequency
from contact_map import plot_utils
import mdtraj as md
import nglview as nv

traj = md.load("data/gsk3b_example.h5")
print(traj)  # to see number of frames; size of system

_ColormakerRegistry()

<mdtraj.Trajectory with 100 frames, 5704 atoms, 360 residues, and unitcells>


In [2]:
topology = traj.topology
yyg = topology.select('resname YYG and element != "H"')
protein = topology.select('protein and element != "H"')

In [3]:
%%time
contacts = ContactFrequency(traj, query=yyg, haystack=protein)

CPU times: user 1.66 s, sys: 10.9 ms, total: 1.67 s
Wall time: 1.68 s


Now we select the MDTraj residue for the drug. Since that is always the same, we'll use it to identify the relevant residue in each of the 

In [4]:
yyg_res = topology.atom(yyg[0]).residue
yyg_res

YYG351

In [5]:
# this takes the residue pair, removes the YYG residue, takes the only thing
# left, and gets its residue index. In detail:
# * `set(res_pair) - set([yyg_res])` takes the pair (order unknown) and 
#   removes yyg_res, leaving the other residue
# * `list(...)[0]` makes it into a list and takes the first (only) element
# * `(...).index` takes the index of the MDTraj Residue object, which is its 
#   order in the file (count from 0)
protein_res_contact = {
    list(set(res_pair) - set([yyg_res]))[0].index: freq
    for res_pair, freq in contacts.residue_contacts.most_common()
}
# now we have a mapping of residue index to the frequency that the residue 
# was in contact with the drug molecule

In [6]:
# if no contacts were made, the count is zero; start with an array of zeros
# (one for each residue)
counts = np.zeros(351)
# any residue whose index that is in the protein_res_contact dictionary should 
# replace the default 0 with the correct freq!
for res, freq in protein_res_contact.items():
    counts[res] = freq
    
# counts is now a list (actually, np.array) with one element for each residue.
# For each residue in the protein, the value at that element is what we want 
# to represent on the figure

Finally, we get to make the picture! NGLView knows how to visualize MDTraj trajectories. We make it so that we're only showing the protein, and then set the colors to be based on the contact frequencies (using our color map of choice).

In [7]:
# convert the counts to colors; NGLView uses web-style hex (prefaced with '#')
colors = plot_utils.hex_colors(counts, cmap='Reds', style='web')
# NGLView color schemes can be defined a list of (color, selection). For
# single-residue selection, select use the string of the number of the residue
# in the PDB, counting from 1. Python counts from 0, so we add 1.
nv_colorscheme = [[color, str(i+1)] for i, color in enumerate(colors)]

In [8]:
# just to show what this looks like; specifically residues with some color!
print(nv_colorscheme[25:30])  

[['#f75d42', '26'], ['#c2161b', '27'], ['#990c13', '28'], ['#ad1116', '29'], ['#e63228', '30']]


In [9]:
nv.color.ColormakerRegistry.add_scheme("contacts", nv_colorscheme)

view = nv.show_mdtraj(traj)
view.clear()
view.add_cartoon(selection='protein', color="contacts")
view

NGLWidget(max_frame=99)

In [ ]:
#view.download_image("GSK3B_contacts.png", trim=True)  # this is how to save that image